In [11]:
import os
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import tqdm as notebook_tqdm

In [12]:
# Initialize model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


In [17]:
# Settings
positive_prompt = "young, friendly, and energetic south east asian college girl"
negative_prompt = "bad_anatomy, 2girl, amputation, disconnected_limbs, cartoon, animate, cgi, render, artwork, illustration, 3d_render, cinema_4d, artstation, octane_render, twisted, UI, underexposed, unnatural, unreal_engine, unrealistic, video_game, deformed_body_features"
image_folders = {
    "Enhance": "./Dataset/Train/2_Enhance",
    "Dzine": "./Dataset/Dzine",
    "Sketch": "./Dataset/Sketch",
}
keywords = ["friendly", "young", "energetic"]  # Key positive prompt keywords

def calculate_clip_score(image_path, positive_prompt, negative_prompt, processor, model, device):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[positive_prompt, negative_prompt], images=image, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    # The higher score is better for positive prompt, the lower is better for negative prompt.
    positive_score = outputs.logits_per_image[0][0].item()
    negative_score = outputs.logits_per_image[0][1].item()
    return positive_score - negative_score # A simple way to combine.  You might want to experiment with different combinations.


In [18]:
# Process images from all specified folders
all_results = {}
for folder_name, image_folder in image_folders.items():
    results = []
    for filename in os.listdir(image_folder):
        if not (filename.endswith(".jpg") or filename.endswith(".png")):  # Adjust as needed
            continue
        image_path = os.path.join(image_folder, filename)
        score = calculate_clip_score(image_path, positive_prompt, negative_prompt, processor, model, device)
        results.append((filename, score))
        print(f"Folder: {folder_name}, Image: {filename}, Combined CLIPScore = {score:.4f}")

    results.sort(key=lambda x: x[1], reverse=True)  # Sort by combined score
    best = results[0] if results else ("N/A", "N/A")
    print(f"\n✅ Best Image in Folder {folder_name}: {best[0]} with Combined CLIPScore = {best[1]:.4f}")
    all_results[folder_name] = {"best_image": best[0], "best_score": best[1], "all_scores": results}

print("\n--- Keyword Analysis (Example - best image from Enhance folder) ---")
if "Enhance" in all_results:
    best_enhance_image = all_results["Enhance"]["best_image"]
    image_path = os.path.join(image_folders["Enhance"], best_enhance_image)
    for keyword in keywords:
        keyword_score = calculate_clip_score(image_path, keyword, negative_prompt, processor, model, device)
        print(f"  Keyword '{keyword}': Score = {keyword_score:.4f}")


Folder: Enhance, Image: Untitled (1).png, Combined CLIPScore = 10.0817
Folder: Enhance, Image: Untitled (12).png, Combined CLIPScore = 8.0175
Folder: Enhance, Image: Untitled (13).png, Combined CLIPScore = 9.7170
Folder: Enhance, Image: Untitled (14).png, Combined CLIPScore = 4.1173
Folder: Enhance, Image: Untitled (15).png, Combined CLIPScore = 4.9786
Folder: Enhance, Image: Untitled (16).png, Combined CLIPScore = 4.1242
Folder: Enhance, Image: Untitled (17).png, Combined CLIPScore = 5.8165
Folder: Enhance, Image: Untitled (18).png, Combined CLIPScore = 6.8701
Folder: Enhance, Image: Untitled (19).png, Combined CLIPScore = 5.4736
Folder: Enhance, Image: Untitled (2).png, Combined CLIPScore = 3.0222
Folder: Enhance, Image: Untitled (20).png, Combined CLIPScore = 5.4736
Folder: Enhance, Image: Untitled (23).png, Combined CLIPScore = 9.9888
Folder: Enhance, Image: Untitled (24).png, Combined CLIPScore = 8.9136
Folder: Enhance, Image: Untitled (25).png, Combined CLIPScore = 9.9457
Folder:

ValueError: Unknown format code 'f' for object of type 'str'

In [19]:
# You can adapt the keyword analysis for other folders as needed.
print("\nOverall Results:")
for folder, data in all_results.items():
    print(f"  Folder: {folder}, Best Image: {data['best_image']}, Best Score: {data['best_score']}")


Overall Results:
  Folder: Enhance, Best Image: Untitled (40).png, Best Score: 13.691920280456543
